In [130]:
# selenium 불러오기 
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from datetime import datetime, timedelta 
from datetime import date as dt
import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [2]:
#driver = webdriver.Chrome(executable_path=r"C:\webdrivers\chromedriver.exe")

In [160]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('window-size=1920x1080')
driver = webdriver.Chrome(executable_path=r'C:\webdrivers\chromedriver.exe', chrome_options=chrome_options)
# TBC 접속하기 
driver.implicitly_wait(3)
driver.get('https://www.tbc.net.tw/Epg/')
wait = WebDriverWait(driver, 5)

<ipython-input-160-e1e093608467>:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=r'C:\webdrivers\chromedriver.exe', chrome_options=chrome_options)


In [60]:
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight/12);")
#driver.execute_script("window.scrollTo(0, window.scrollY + 200);")

In [132]:
def update_date(driver,start_date): #ex)2021/02/28
  
  #change hour to initial 00:00
  hour_pick = Select(driver.find_element_by_id("SearchParameter_Hour"))
  hour_pick.select_by_visible_text("00:00") 
  #change date
  date_pick = driver.find_element_by_id('SearchParameter_StartDate')
  upinpt = driver.execute_script("arguments[0].setAttribute('value','{a_date}')".format(a_date = start_date), date_pick)
  #driver.execute_script("($'#SearchParameter_Hour').val('0');")
  #driver.execute_script("($'#SearchParameter_StartDate').val({a_date});".format(a_date = date))
  update_click = driver.find_element_by_xpath('//*[@id="epgForm"]/div[1]/div[4]/a[2]')
  # result = driver.execute_script("arguments[0].click();", update_click)
  print('Date is',start_date)
  print('Hour is 00:00',)
  print("Succesfuly updated")
  upclic = driver.execute_script("arguments[0].click();", update_click)
  return (start_date, upinpt,"00:00",upclic)

In [133]:
def next_tmLine(start_hr, start_date,end_date):
  
  opt_seletd = Select(driver.find_element_by_id("SearchParameter_Hour"))
  seletd_tm = opt_seletd.first_selected_option
  print("current time starting "
        "point :{}".format(seletd_tm.text))
  prsent_tm = seletd_tm.text
  if (prsent_tm != "18:00"):
    print("you are eligible to increment hours")
    #to convert string to time format HH:MM
   # start_tm = #"00:00"
    weighted_tm = 6
    int_tm = int(start_hr.rsplit(':', 1)[0])
    added_tm = int_tm + weighted_tm
    start_hr = "{}:00".format(str(added_tm).zfill(2))
    print("Date is",start_date)
    print("new start hour is :",start_hr)
   
    opt_seletd.select_by_visible_text(start_hr) 
    update_click = driver.find_element_by_xpath('//*[@id="epgForm"]/div[1]/div[4]/a[2]')
    uphr = driver.execute_script("arguments[0].click();", update_click)
    print("Succesfuly updated")
      
    return (prsent_tm, start_hr,str(start_date), uphr, uphr)
  else:
    print("pls, use update_date_function instead")
    #start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
    start_date = start_date.strftime("%Y/%m/%d")
    #start_date += datetime.timedelta(days=1)
    start_date = str(start_date)
    print("Date is",start_date)
    out = update_date(driver,start_date)
    print("out[0]:",out[0])
    return (out[0],"00:00", out[0])


In [5]:
# hr = '00:00'
# init_day = '2021/03/01'
# end_day = '2021/03/10'
# update_date(driver, init_day)
# #next_tmLine(hp,tp,fp)
# while init_day < end_day:
#  get = next_tmLine(hr,init_day,end_day)
#  init_day = get[1]
#  end_day = get[2]

In [134]:
#@markdown check each day info
def today_info():
  """Wait and Check the len of EPG and the group of corresponding CNT """
  try:  
    epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
    cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul')))
    if (len(epg) != len(cnt)):  
  # we can pass the message in the raise statement  
      raise ValueError("the number of channels and contents lists don't match!!") 
    else:
      #print('the num of channels and the num of lists of contents on the table match! Keep crawling!') 
      #print("the each num of channel and lists of contents is {}".format(len(epg)))
      print("channel and contents nm {} matches".format(len(epg)))
  except ValueError as e:  
      print(e) 
  return (None)

In [161]:
def get_Epg_info():

    epg_info = []
    
    epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
    cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul')))
    count = 0
    for i in range(2,257): #range(1,259) #range(len(epg))
      # if (i=18):
      # driver.execute_script("window.scrollTo(0, document.body.scrollHeight/12);")    
      tv_name = epg[i].text.split('\n') #get the channel name //*[@id="content"]/ul[3]/li[3]/a
      print("start:",tv_name)
      if int(tv_name[0]) < 300 : #tv_name[0] != '西洋排行新曲'
        #a_ctn_list = wait.until(
        #  EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)))) # match to the corresponding CH
        a_ctn_list = cnt[i].find_elements_by_xpath('//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)) # match to the corresponding CH#
        ctn_info = []
        for j in range(len(a_ctn_list)): #get the each info from the correspoing lists of contents
          a_ctn = webdriver.ActionChains(driver).move_to_element(a_ctn_list[j]).click() #--》이것을 아래의 element_to_be_clickable로 바뀌기
          a_ctn.perform()
          #a_ctn = wait.until(
          #  EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/ul[{:d}]/li[{:d}]/a'.format(i+1,j+1))))
          #a_ctn.click()
          try: # if one of info is empty, we just leave it as a blank
            #driver.implicitly_wait(3)
            # ctn_info.append(tv_name +
            #             [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_name"]'))).text] +
            #             wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_moreinfo"]/p'))).text.split('\n') +
            #             [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_desc"]'))).text.replace("\n", " ")] +
            #             [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_date"]'))).text.replace("\n", " ")] +  #--> 다 wait으로 바꾸기
            #             wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_time"]'))).text.split()
            #             )
            
            ctn_info.append(tv_name +
              [driver.find_element_by_xpath('//*[@id="prg_name"]').text] +
              driver.find_element_by_xpath('//*[@id="prg_moreinfo"]/p').text.split('\n') +
              [driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")] +
              [driver.find_element_by_xpath('//*[@id="prg_date"]').text.replace("\n", " ")] +  #--> 다 wait으로 바꾸기
              driver.find_element_by_xpath('//*[@id="prg_time"]').text.split()
              )
          
          except (NoSuchElementException,TimeoutException) as e :
            # Just append a None or ""
            ctn_info.append(['']+['']+['']+['']+['']+[''])
          
          # epg_info.append({'channel': tv_name,
          #                   'prog': list(zip(*ctn_info))[0],
          #                   'genre': list(zip(*ctn_info))[1],
          #                   'desc': list(zip(*ctn_info))[2],
          #                   'time': list(zip(*ctn_info))[4]
          #                 })    
          # close_btn = '//*[@id="epg_popup"]/div/div[1]/div[3]/a'
          # close_btn_ele = driver.find_element_by_xpath(close_btn)
          # close_pop = webdriver.ActionChains(driver).move_to_element(close_btn_ele).click()
          # close_pop.perform()
          epg_info.extend(ctn_info)                                    #//*[@id="epg_popup"]/div/div[1]/div[3]/a
          #try:
          element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="epg_popup"]/div/div[1]/div[3]/a')))
          element.click()
            
          # except (TimeoutException) as e :
          #   print("tried other way to close") 
          #   print("Exception has been thrown. " + str(e))
            

            # epg_info.extend(ctn_info)                                    #//*[@id="epg_popup"]/div/div[1]/div[3]/a
            # element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="epg_popup"]/div/div[1]/div[2]/a')))
            # element.click()        
            # print("tried other way to close")   
          #driver.implicitly_wait(3)
        
        print("done:",tv_name) 
        count +=1
        if count % 10 ==  0:
          print("did ten")
        else:
          print("count:",count)
      # if i % 2 == 0:
      #   driver.execute_script("window.scrollTo(0, window.scrollY + 200);")
      else:
      #   #//*[@id="epg_music_popup"]/div/div[1]/div[2]/a
      #   # element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="epg_popup"]/div/div[1]/div[2]/a')))
      #   # element.click()
      #   close_btn = '//*[@id="epg_popup"]/div/div[1]/div[2]/a'
      #   close_btn_ele = driver.find_element_by_xpath(close_btn)
      #   close_pop = webdriver.ActionChains(driver).move_to_element(close_btn_ele).click()
      #   close_pop.perform()
        return (epg_info)
    


In [162]:
hr = '00:00'
init_day = '2021/03/02'
end_day = '2021/03/03'
update_date(driver, init_day)
#next_tmLine(hp,tp,fp)
EPG_per = []
while init_day < end_day:
 EPG_per.extend(get_Epg_info())
 get = next_tmLine(hr,init_day,end_day)
 
 hr = get[1]
 init_day = get[2]
 

Date is 2021/03/02
Hour is 00:00
Succesfuly updated
start: ['6', '民視無線台']
done: ['6', '民視無線台']
count: 1
start: ['7', 'TBC桃園生活台']
done: ['7', 'TBC桃園生活台']
count: 2
start: ['8', '台視']
done: ['8', '台視']
count: 3
start: ['9', '大愛電視台']
done: ['9', '大愛電視台']
count: 4
start: ['10', '中視']
done: ['10', '中視']
count: 5
start: ['11', '人間衛視']
done: ['11', '人間衛視']
count: 6
start: ['12', '華視']
done: ['12', '華視']
count: 7
start: ['13', '公共電視']
done: ['13', '公共電視']
count: 8
start: ['14', '公視台語台']
done: ['14', '公視台語台']
count: 9
start: ['15', '好消息頻道']
done: ['15', '好消息頻道']
did ten
start: ['16', '原住民頻道']
done: ['16', '原住民頻道']
count: 11
start: ['17', '客家電視台']
done: ['17', '客家電視台']
count: 12
start: ['18', '國家地理頻道']
done: ['18', '國家地理頻道']
count: 13
start: ['19', 'Discovery']
done: ['19', 'Discovery']
count: 14
start: ['20', 'TLC旅遊生活頻道']
done: ['20', 'TLC旅遊生活頻道']
count: 15
start: ['21', '動物星球']
done: ['21', '動物星球']
count: 16
start: ['22', '卡通頻道']
done: ['22', '卡通頻道']
count: 17
start: ['23', '迪士尼頻道']
done: ['23'

In [23]:
test= get_Epg_info()

In [164]:

#@title Save the Genres variables
#save variables
import pickle
with open('EPG_per_0203_0321','wb') as f:
  pickle.dump(EPG_per, f)

In [163]:
EPG_per_0203_0321 = EPG_per

In [165]:
import pandas as pd
df = pd.DataFrame(EPG_per_0203_0321)
#df.columns =['prog_name','genre','desc']
print(len(df.index))

27024


In [166]:
df

,0,1,2,3,4,5,6
0,6,民視無線台,多情城市:372,類別：亞洲連續劇 導演：朱莉莉;蕭大陸;張志鴻,述說三個拜把兄弟如何赤手空拳打天下，藉由上一代的愛情故事串連兩代之間共同的情感與記憶，深度探...,2021/03/01,23:15~01:30
1,6,民視無線台,多情城市:372,類別：亞洲連續劇 導演：朱莉莉;蕭大陸;張志鴻,述說三個拜把兄弟如何赤手空拳打天下，藉由上一代的愛情故事串連兩代之間共同的情感與記憶，深度探...,2021/03/01,23:15~01:30
2,6,民視無線台,元氣加油站,類別：知性（知識）,享受樂活人生!將健康概念融入衣食住行生活起居之中，簡單有效率的照顧全家人的健康，西醫觀點的家...,2021/03/02,01:30~02:00
3,6,民視無線台,多情城市:372,類別：亞洲連續劇 導演：朱莉莉;蕭大陸;張志鴻,述說三個拜把兄弟如何赤手空拳打天下，藉由上一代的愛情故事串連兩代之間共同的情感與記憶，深度探...,2021/03/01,23:15~01:30
4,6,民視無線台,元氣加油站,類別：知性（知識）,享受樂活人生!將健康概念融入衣食住行生活起居之中，簡單有效率的照顧全家人的健康，西醫觀點的家...,2021/03/02,01:30~02:00
...,...,...,...,...,...,...,...
27019,258,靖天卡通台,ABC小熊:24,類別：卡通/兒童,內容以數數字、找字母，激發孩童對於語文及數理的基本觀念，透過可愛的小熊、水獺、刺蝟、青蛙、企...,2021/03/02,21:30~22:00
27020,258,靖天卡通台,超級總動員第12季:11,類別：卡通/兒童、綜藝/娛樂,對於海綿寶寶和萌學園你了解多少呢？你是海綿寶寶達人嗎？還是你是萌學園達人？或者你是深藏不露的...,2021/03/02,22:00~22:30
27021,258,靖天卡通台,超級總動員第12季:11,類別：卡通/兒童、綜藝/娛樂,對於海綿寶寶和萌學園你了解多少呢？你是海綿寶寶達人嗎？還是你是萌學園達人？或者你是深藏不露的...,2021/03/02,22:30~23:00
27022,258,靖天卡通台,YOYO好好玩:8,類別：卡通/兒童、綜藝/娛樂,YOYO好好玩,2021/03/02,23:00~23:30


In [ ]:
a_l = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/ul[3]/li[2]/a')))
a_ctn = webdriver.ActionChains(driver).move_to_element(a_l).click()
a_ctn.perform()

element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="epg_popup"]/div/div[1]/div[3]/a')))
element.click()

#@title Create CLOSE btn!!
close_btn = '//*[@id="epg_popup"]/div/div[1]/div[3]/a'
close_btn_ele = driver.find_element_by_xpath(close_btn)
close_pop = webdriver.ActionChains(driver).move_to_element(close_btn_ele).click()
close_pop.perform()



In [ ]:
epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul')))
print(epg[1].text.split('\n')[0])    
print(int(epg[1].text.split('\n')[0]))    
tv_name = epg[i].text.split('\n')
print(tv_name)  

a_ctn_list = cnt[0].find_elements_by_xpath('//*[@id="content"]/ul[{:d}]/li/a'.format(0+1)) # match to the corresponding CH#
print(a_ctn_list)
ctn_info = []
 #get the each info from the correspoing lists of contents
a_ctn = webdriver.ActionChains(driver).move_to_element(a_ctn_list[0]).click() #--》이것을 아래의 element_to_be_clickable로 바뀌기
a_ctn.perform()

print(driver.find_element_by_xpath('//*[@id="prg_name"]').text)

# ctn_info.append(tv_name +
#     [driver.find_element_by_xpath('//*[@id="prg_name').text] +
#     driver.find_element_by_xpath('//*[@id="prg_moreinfo"]/p').text.split('\n') +
#     [driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")] +
#     [driver.find_element_by_xpath('//*[@id="prg_date"]').text.replace("\n", " ")] +  #--> 다 wait으로 바꾸기
#     driver.find_element_by_xpath('//*[@id="prg_time"]').text.split()
#     )

3
3
['3', '南桃園公用頻道']
[<selenium.webdriver.remote.webelement.WebElement (session="ef903bfcee66c96ec66131d5cb09722a", element="bdf919c6-0f81-44b5-8956-05fb56f10d96")>, <selenium.webdriver.remote.webelement.WebElement (session="ef903bfcee66c96ec66131d5cb09722a", element="ca7b7044-36b7-4c41-9e08-5efbd9a09b6c")>, <selenium.webdriver.remote.webelement.WebElement (session="ef903bfcee66c96ec66131d5cb09722a", element="7b36030e-fbee-4ede-9748-07834a211b63")>, <selenium.webdriver.remote.webelement.WebElement (session="ef903bfcee66c96ec66131d5cb09722a", element="3da2c11a-86e5-407a-a24e-51bb5825314d")>, <selenium.webdriver.remote.webelement.WebElement (session="ef903bfcee66c96ec66131d5cb09722a", element="8e6acae4-7188-4f17-8158-d7127e8c578b")>, <selenium.webdriver.remote.webelement.WebElement (session="ef903bfcee66c96ec66131d5cb09722a", element="791f8124-73d3-4a39-b8c3-5a79a291884f")>, <selenium.webdriver.remote.webelement.WebElement (session="ef903bfcee66c96ec66131d5cb09722a", element="e6f12569-e9

def get_Epg_info():

    epg_info = []
    
    epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
    cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul')))
    count = 0
    for i in range(159,257): #range(1,259) #range(len(epg))
      
      # if (i=18):
      #   driver.execute_script("window.scrollTo(0, document.body.scrollHeight/12);")
                
      tv_name = epg[i].text.split('\n') #get the channel name //*[@id="content"]/ul[3]/li[3]/a
      if tv_name == "西洋排行新曲":
        break
    else:
        
      #a_ctn_list = wait.until(
      #  EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)))) # match to the corresponding CH
      a_ctn_list = cnt[i].find_elements_by_xpath('//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)) # match to the corresponding CH#
      
      ctn_info = []
      for j in range(len(a_ctn_list)): #get the each info from the correspoing lists of contents
        a_ctn = webdriver.ActionChains(driver).move_to_element(a_ctn_list[j]).click() #--》이것을 아래의 element_to_be_clickable로 바뀌기
        a_ctn.perform()
        #a_ctn = wait.until(
        #  EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/ul[{:d}]/li[{:d}]/a'.format(i+1,j+1))))
        #a_ctn.click()
        try: # if one of info is empty, we just leave it as a blank
          #driver.implicitly_wait(3)
          # ctn_info.append(tv_name +
          #             [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_name"]'))).text] +
          #             wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_moreinfo"]/p'))).text.split('\n') +
          #             [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_desc"]'))).text.replace("\n", " ")] +
          #             [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_date"]'))).text.replace("\n", " ")] +  #--> 다 wait으로 바꾸기
          #             wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_time"]'))).text.split()
          #             )
          ctn_info.append(tv_name +
            [driver.find_element_by_xpath('//*[@id="prg_name').text] +
            driver.find_element_by_xpath('//*[@id="prg_moreinfo"]/p').text.split('\n') +
            [driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")] +
            [driver.find_element_by_xpath('//*[@id="prg_date"]').text.replace("\n", " ")] +  #--> 다 wait으로 바꾸기
            driver.find_element_by_xpath('//*[@id="prg_time"]').text.split()
            )
        
        except (NoSuchElementException) as e :
          # Just append a None or ""
          ctn_info.append(['']+['']+['']+['']+['']+[''])
        
        # epg_info.append({'channel': tv_name,
        #                   'prog': list(zip(*ctn_info))[0],
        #                   'genre': list(zip(*ctn_info))[1],
        #                   'desc': list(zip(*ctn_info))[2],
        #                   'time': list(zip(*ctn_info))[4]
        #                 })    
        epg_info.extend(ctn_info)                                    #//*[@id="epg_popup"]/div/div[1]/div[3]/a
        element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="epg_popup"]/div/div[1]/div[3]/a')))
        element.click()
        #driver.implicitly_wait(3) 
      count +=1
      if count % 10 ==  0:
        print("did ten")
      else:
        print("count:",count)
      # if i % 2 == 0:
      #   driver.execute_script("window.scrollTo(0, window.scrollY + 200);")

    return (epg_info)